# Lesson 4 - BOLDsequence

The `BOLDsequence` module allows us to take our dephasing time series and convert it to a signal using a pulse sequence. To begin we first import the module as follows (and NumPy for array creation, matplotlib for plotting and tqdm for progressbars):

In [ ]:
from BOLDswimsuite import BOLDgeometry, BOLDspins, BOLDsequence
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # for progress bars

Similarily to the previous lesson, we start by creating a randomly generated continuous voxel.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel3D.from_random(
    size=0.1,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.002, 0.003, 0.004], 
        'artery': [0.003, 0.004, 0.005]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={ # we use impermeable vessels here as permeable vessels require a very high spin count to produce a relatively noise-free signal output
        'vein': 0, 
        'artery': 0
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)

print(f'Number of vessels: {len(random_continuous_voxel.vessels)}')

We now create a 3D spins object with our new geometry.

In [ ]:
num_spins = 10_000
dt=0.2 #we will use a constant 0.2ms time step

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

Now we can create a `SpinSequence` object. It has the following arguments:
- spins : BOLDspins.Spins, the spins object which will be used in the sequence.
- pulse_time_indices : List[int], list of the number of time steps before each pulse.
- pulse_angles : List[float], list of angle of each pulse (radians). Must be the same length as `pulse_time_indices`
- pulse_axes : List[List[float]], list of axes around which to rotate the spins, in polar coordinates (radians). Each axis in the list is a 2 item list with the form `[phi,theta]`. For example, a pulse on the x-axis will be represented as `[np.pi/2, 0]` and a pules on the y-axis will be represented as `[np.pi/2, np.pi/2]`.

In [ ]:
sequence = BOLDsequence.SpinSequence(
    spins=spins,
    pulse_time_indices=[0, 50],                     #[0ms       , 10ms]
    pulse_angles=[np.pi/2, np.pi],                  #[90 degrees, 180 degrees]
    pulse_axes=[[np.pi/2, np.pi/2], [np.pi/2, 0]]   #[y-axis    , x-axis]
)

We can now obtain the signal for the first time step. Since we have a y-axis 90 degree pulse at time step 0, it will automatically be applied. We use the `step` method to do this, which requires the step length `dt` in ms. The `SpinSequence` object automatically takes care of taking steps with the `Spins` object every time we step through the sequence:

In [ ]:
sequence.step(dt=0.2)

To get the signal, we use the `get_signals` method, which returns the total signal, EV signal and IV signal. It only has one argument:
- cplx : bool, if True, will return the complex signal, if False returns the magnitude of the signal. Default is False.

In [ ]:
eviv, ev, iv = sequence.get_signals(cplx=False)

print(f'Total signal: {eviv}')
print(f'EV signal: {ev}')
print(f'IV signal: {iv}')

Now if we want to calculate the rest of the signal, we can use a simple 'for' loop, and some arrays to store the results.

>Note: we are using tqdm to add a progress bar to the for loop, which helps track the progress and gives an estimate of the remaining time.

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

# 3 arrays to store the signals
eviv = np.zeros(num_steps) 
ev = np.zeros(num_steps)
iv = np.zeros(num_steps)

# for loop to repeat the Monte Carlo steps and signal calculation
for j in tqdm(range(num_steps)): # use just `for j in range(num_steps)` to remove the progress bar

    # we advance the pulse sequence by one step, which will also advance the spins by a step
    sequence.step(dt=0.2)

    # we retrieve the signals and store them in the arrays
    eviv[j], ev[j], iv[j] = sequence.get_signals(cplx=False)

Now we can plot the signals using matplotlib.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()

We can also use the `walk` method of the sequence object to automatically walk through `num_steps` steps, although this option is a bit less flexible. To show this we will create new `Spins` and `SpinSequence` objects to start fresh:

In [ ]:
num_spins = 10_000
dt=0.2

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed will provide the same result
)

sequence = BOLDsequence.SpinSequence(
    spins=spins,
    pulse_time_indices=[0, 50], # 50*0.2ms = 10ms
    pulse_angles=[np.pi/2, np.pi],
    pulse_axes=[[np.pi/2, np.pi/2], [np.pi/2, 0]]
)

We now use the `walk` method to advance through the entire time range and produce the 3 signal arrays. The function also has a `progressbar` option which when set to `True` will display a progress bar:

In [ ]:
eviv, ev, iv = sequence.walk(
    dt=0.2,
    num_steps=num_steps,
    progressbar=True
)

We can then plot the signals:

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()

### No Diffusion Simulations

If we do not want to model diffusion in our simulation, rather than setting the ADC to 0, we can use the `Sequence` object (not the `SpinSequence` object as previously). This will allow us to step through the sequence without stepping through the spins, which will have the effect of no diffusion. Doing it this way is faster than setting the ADC to 0 as it prevents the simulation from recalculating the dephasing at each time step. We first create our spins and sequence objects.

In [ ]:
num_spins = 10_000
dt=0.2

spins = BOLDspins.Spins3D(
    ADC=0.001,
    num_spins=num_spins,
    geometry=random_continuous_voxel,
    dt=dt,
    IV=True,
    seed=0 #repeating with the same seed with provide the same result
)

sequence = BOLDsequence.Sequence(
    sample_shape=num_spins,
    pulse_time_indices=[0, 50], # 50*0.2ms = 10ms
    pulse_angles=[np.pi/2, np.pi],
    pulse_axes=[[np.pi/2, np.pi/2], [np.pi/2, 0]]
)

>Note: the `Sequence` object takes in `sample_shape` and not `spins` as an argument when creating the object. This makes it more flexible, allowing us in this case to prevent stepping through the spins. For our purposes, `sample_shape` is just the number of spins (`num_spins`).

Then we use the same 'for' loop method as with the `SpinSequence`. Here we must pass the `step` method the phase samples (`phase`) and intravascular state of each phase sample (`is_IV`). Both of these can be taken from the `Spins` object.

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

# 3 arrays to store the signals
eviv = np.zeros(num_steps)
ev = np.zeros(num_steps)
iv = np.zeros(num_steps)

phase = spins.phase
is_IV = spins.vessel_indices>0 # any vessel index greater than 0 is intravascular

for j in tqdm(range(num_steps)): # use just `for i in range(num_steps)` to remove the progress bar
    sequence.step(phase=phase, is_IV=is_IV)
    eviv[j], ev[j], iv[j] = sequence.get_signals(cplx=False)

We again plot the result, and see that the resulting signal indeed has the characteristics of no-diffusion simulations, such as perfect refocusing at the spin echo.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()